# SQL Question Answering Benchmarking: Chinook

这里我们将介绍如何在 SQL 数据库上对问题回答任务进行性能基准测试。

强烈建议您启用跟踪来进行任何评估/基准测试。有关跟踪是什么以及如何设置它的说明，请参见 [这里](https://langchain.readthedocs.io/en/latest/tracing.html)。

In [28]:
# Comment this out if you are NOT using tracing
import os
os.environ["LANGCHAIN_HANDLER"] = "langchain"

## 加载数据

首先，让我们加载数据。

In [7]:
from langchain.evaluation.loading import load_dataset
dataset = load_dataset("sql-qa-chinook")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /Users/harrisonchase/.cache/huggingface/datasets/LangChainDatasets___json/LangChainDatasets--sql-qa-chinook-7528565d2d992b47/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
dataset[0]

{'question': 'How many employees are there?', 'answer': '8'}

## 设置链
这使用了 Chinook 数据库示例。
按照 https://database.guide/2-sample-databases-sqlite/ 上的说明进行设置，将 `.db` 文件放在此存储库根目录下的 notebooks 文件夹中。

请注意，这里我们载入了一个简单的链。如果要尝试更复杂的链或者代理，请以其他方式创建 `chain` 对象。

In [1]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

In [3]:
db = SQLDatabase.from_uri("sqlite:///../../../notebooks/Chinook.db")
llm = OpenAI(temperature=0)

现在我们可以创建一个 SQL 数据库链。

In [14]:
chain = SQLDatabaseChain.from_llm(llm, db, input_key="question")

## 进行预测

首先，我们可以逐个数据点进行预测。这样做可以详细探索输出结果，而且比多个数据点运行更具成本效益。

In [27]:
chain(dataset[0])

{'question': 'How many employees are there?',
 'answer': '8',
 'result': ' There are 8 employees.'}

## 进行许多预测
现在我们可以进行预测。请注意，我们添加了 try-except，因为这个链有时会出现错误（如果 SQL 书写不正确等）。

In [19]:
predictions = []
predicted_dataset = []
error_dataset = []
for data in dataset:
    try:
        predictions.append(chain(data))
        predicted_dataset.append(data)
    except:
        error_dataset.append(data)

## 评估性能
现在我们可以评估预测结果。我们可以使用语言模型来以程序化方式对其进行评分。

In [21]:
from langchain.evaluation.qa import QAEvalChain

In [22]:
llm = OpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate(predicted_dataset, predictions, question_key="question", prediction_key="result")

我们可以将分级输出添加到 `predictions` 字典中，然后获取等级统计信息。

In [23]:
for i, prediction in enumerate(predictions):
    prediction['grade'] = graded_outputs[i]['text']

In [24]:
from collections import Counter
Counter([pred['grade'] for pred in predictions])

Counter({' CORRECT': 3, ' INCORRECT': 4})

我们还可以对不正确的示例进行数据点过滤和查看。

In [25]:
incorrect = [pred for pred in predictions if pred['grade'] == " INCORRECT"]

In [26]:
incorrect[0]

{'question': 'How many employees are also customers?',
 'answer': 'None',
 'result': ' 59 employees are also customers.',
 'grade': ' INCORRECT'}